# Test wind forecast with barometric pressure

In [1]:
# install the modules on the OS
#!pip install influxdb
#!pip3 install ipynb

# import the modules
import pandas as pd
import datetime as dt
import numpy as np
import influxdb
import os
import math
import scipy.stats as st
import plotly.graph_objects as go

from ipynb.fs.full.config_and_functions import *

In [2]:
def get_air_temperature_forecast_values():
    
    # get datetime of last entry
    datetime_of_last_entry = get_datetime_of_last_entry()
    
    # Get data for air temperature prediction
    query = """SELECT
            last(air_temperature) AS last_air_temperature,
            last(dew_point) AS last_dew_point,
            last(humidity) AS last_humidity,
            last(barometric_pressure_qfe) AS last_barometric_pressure_qfe
            FROM "meteorology"."autogen"."{}" """.format(station_name)

    df_last = pd.DataFrame(client.query(query)[station_name])
    
    last_air_temperature = float(df_last["last_air_temperature"].values)
    last_dew_point = float(df_last["last_dew_point"].values)
    last_humidity = float(df_last["last_humidity"].values)
    last_barometric_pressure_qfe = float(df_last["last_barometric_pressure_qfe"].values)

    #print(last_air_temperature)
    #print(last_dew_point)
    #print(last_humidity)
    #print(last_barometric_pressure_qfe)

    # find similar conditions, stop if something found
    # "simulate" HAVING clause
    len_of_df = 0
    factor = 0.05
    while len_of_df <= 20:
        
        query = """SELECT
                (air_temperature),
                (dew_point),
                (humidity),
                (barometric_pressure_qfe)
                FROM "meteorology"."autogen"."{}"
                WHERE
                    (air_temperature > {} AND air_temperature < {})
                    AND
                    (dew_point > {} AND dew_point < {})
                    AND
                    (humidity > {} AND humidity < {})
                    AND
                    (barometric_pressure_qfe > {} AND barometric_pressure_qfe < {})
                    AND
                    (time < now() - 3d)""".format(
                            station_name,
                            last_air_temperature - factor * 1,
                            last_air_temperature + factor * 1,
                            last_dew_point - factor * 20,
                            last_dew_point + factor * 20,
                            last_humidity - factor * 20,
                            last_humidity + factor * 20,
                            last_barometric_pressure_qfe - factor * 40,
                            last_barometric_pressure_qfe + factor * 40)

        df = pd.DataFrame(client.query(query)[station_name])
        len_of_df = len(df)
        factor += 0.1
        if factor == 10:
            break
    

    #print('df:\n', df, '\n---', 'len(df)', len(df))
    

    # loop in widening day span, break if span is 60 (2 x 30) days
    days_span = 0
    len_of_df_f = 0
   
    while len_of_df_f <= 10:
        
        df_f = df.copy()
        df_f = get_data_in_current_time_period(df_f, days_span, days_span) # Filter for current time period

        if days_span == 30:
            break
            
        len_of_df_f = len(df_f)
        days_span += 1

        
    #print('len_of_df_f: {}'.format(len_of_df_f))
    #print('df_f:\n', df_f, '\n---')
    
    
    # loop in widening timespan, break if dataframe has length of 1
    minutes_span = 0
    len_of_df_ff = 0
    while len_of_df_ff <= 0:

        df_ff = df_f.copy()
        df_ff = df_ff.between_time(
            ((datetime_of_last_entry + dt.timedelta(minutes=-minutes_span)).strftime('%H:%M'))[0],
            ((datetime_of_last_entry).strftime('%H:%M'))[0]
        )
        if len(df_ff) == 1: break

        df_ff = df_ff.between_time(
            ((datetime_of_last_entry).strftime('%H:%M'))[0],
            ((datetime_of_last_entry + dt.timedelta(minutes=minutes_span)).strftime('%H:%M'))[0]
        )
        if len(df_ff) == 1: break

        len_of_df_ff = len(df_ff)
        minutes_span += 10

    #print('minutes_span: {}'.format(minutes_span))
    #print('len_of_df_ff: {}'.format(len_of_df_ff))
    #print('df_ff:\n', df_ff, '\n---')


    # go 2 hours back and 6 forward, get min and max values
    datetime_start = (df_ff.iloc[0:].index[0]
                          .replace(
                              hour=datetime_of_last_entry.hour[0],
                              minute=datetime_of_last_entry.minute[0]
                          )
                     ) 
    datetime_end = (df_ff.iloc[0:].index[0] + dt.timedelta(days=3)) 

    #print('datetime_start: {}'.format(datetime_start))
    #print('datetime_end: {}'.format(datetime_end))
    
    query = """SELECT
                (air_temperature)
                FROM "meteorology"."autogen"."{}"
                WHERE
                    (time >= '{}' AND time <= '{}')""".format(
                        station_name,
                        datetime_start.tz_localize(tz = None),
                        datetime_end.tz_localize(tz = None)
    )
    
    df_final = pd.DataFrame(client.query(query)[station_name])
    
    starting_datetime = df_ff.iloc[0:].index[0]
    forecast_values = []
    forecast_values.append(df_final.loc[starting_datetime + dt.timedelta(hours=2)].values)
    forecast_values.append(df_final.loc[starting_datetime + dt.timedelta(hours=4)].values)
    forecast_values.append(df_final.loc[starting_datetime + dt.timedelta(hours=8)].values)
    forecast_values.append(df_final.loc[(starting_datetime + dt.timedelta(hours=24)).replace(hour=12, minute=0)].values)
    forecast_values.append(df_final.loc[(starting_datetime + dt.timedelta(hours=48)).replace(hour=12, minute=0)].values)


    #print('2h: {}'.format(forecast_values[0]))
    #print('4h: {}'.format(forecast_values[1]))
    #print('8h: {}'.format(forecast_values[2]))
    #print('morgen mittag: {}'.format(forecast_values[3]))
    #print('übermorgen mittag: {}'.format(forecast_values[4]))
    
    return forecast_values

get_air_temperature_forecast_values()

[array([1.9]), array([2.]), array([0.7]), array([-0.6]), array([-4.7])]